In [24]:
# Import Stuff
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
from imgextract import Extractor
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from multiprocessing import Process
from IPython.display import clear_output

In [28]:
# All Parameters required for training are declared over here
# Frequency of Image Capturing
FRAME_SKIP = 2
# Frame Size
FRAME_SIZE = (150,150)

In [25]:
# Dataset
!rm -r A-Dataset-for-Automatic-Violence-Detection-in-Videos/
!git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos

Cloning into 'A-Dataset-for-Automatic-Violence-Detection-in-Videos'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 367 (delta 0), reused 2 (delta 0), pack-reused 364
Receiving objects: 100% (367/367), 1.02 GiB | 21.39 MiB/s, done.
Checking out files: 100% (355/355), done.


In [26]:
!rm -r Data
!mkdir Data
!mkdir -p ./Data/Video/Violent
!mkdir -p ./Data/Video/NonViolent
!cp -a ./A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/violent/cam1/. ./Data/Video/Violent/
!cp -a ./A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/non-violent/cam1/. ./Data/Video/NonViolent/
clear_output()
!mkdir -p ./Data/Training/V
!mkdir -p ./Data/Training/NV

In [29]:
def thread_1():
    ext = Extractor(FRAME_SIZE, FRAME_SKIP)
    for i in range(60):
        path = f"./Data/Video/Violent/{i+1}.mp4"
        print(f"Processing Violent Vid-{i}")
        ext.extract(path, 'V')
    print("Violent Extracted")

def thread_2():
    ext = Extractor(FRAME_SIZE, FRAME_SKIP)
    for i in range(60):
        path = f"/content/Data/Video/NonViolent/{i+1}.mp4"
        print(f"Processing NonViolent Vid-{i}")
        ext.extract(path, 'NV')
    print("Non-Violent Extracted")

In [30]:
# Violent Extraction
t1 = Process(target=thread_1, args=())
t2 = Process(target=thread_2, args=())

t1.start() 
t2.start()
# NonViolent Extraction

t1.join()
t2.join()
print("Complete")

Processing Violent_0
Processing NonViolent_0
Processing Violent_1
Processing NonViolent_1
Processing Violent_2
Processing NonViolent_2
Processing Violent_3
Processing NonViolent_3
Processing Violent_4
Processing NonViolent_4
Processing Violent_5
Processing NonViolent_5
Processing Violent_6
Processing NonViolent_6
Processing NonViolent_7
Processing Violent_7
Processing NonViolent_8
Processing Violent_8
Processing NonViolent_9
Processing Violent_9
Processing NonViolent_10
Processing NonViolent_11
Processing Violent_10
Processing NonViolent_12
Processing Violent_11
Processing Violent_12
Processing NonViolent_13
Processing Violent_13
Processing NonViolent_14
Processing Violent_14
Processing NonViolent_15
Processing Violent_15
Processing NonViolent_16
Processing Violent_16
Processing Violent_17
Processing NonViolent_17
Processing Violent_18
Processing NonViolent_18
Processing NonViolent_19
Processing Violent_19
Processing NonViolent_20
Processing Violent_20
Processing NonViolent_21
Processi

In [36]:
base_dir='./Data'
train_dir=os.path.join(base_dir,'Training')
train_violent_dir =os.path.join(train_dir, 'V' )
train_nonviolent_dir=os.path.join(train_dir,'NV')


In [38]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=40,width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,horizontal_flip=True, fill_mode='nearest')

In [39]:

train_generator = train_datagen.flow_from_directory(train_dir,color_mode="rgb", target_size = FRAME_SIZE,batch_size=20,classes=['NV','V'], class_mode='binary', shuffle=True)

Found 10224 images belonging to 2 classes.


In [40]:
model= tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Dropout(0.5),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(512, activation='relu'),
       tf.keras.layers.Dense(1,activation ='sigmoid')

])

In [41]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [45]:
model1=model.fit(train_generator,steps_per_epoch=50, epochs=30)

Epoch 1/30
50/50 [==============================] - 6s 111ms/step - loss: 0.3255 - accuracy: 0.8550
Epoch 2/30
50/50 [==============================] - 6s 112ms/step - loss: 0.3168 - accuracy: 0.8630
Epoch 3/30
50/50 [==============================] - 6s 114ms/step - loss: 0.3077 - accuracy: 0.8730
Epoch 4/30
50/50 [==============================] - 7s 138ms/step - loss: 0.2901 - accuracy: 0.8760
Epoch 5/30
50/50 [==============================] - 6s 113ms/step - loss: 0.3141 - accuracy: 0.8710
Epoch 6/30
50/50 [==============================] - 6s 113ms/step - loss: 0.3354 - accuracy: 0.8620
Epoch 7/30
50/50 [==============================] - 6s 114ms/step - loss: 0.2790 - accuracy: 0.8910
Epoch 8/30
50/50 [==============================] - 6s 116ms/step - loss: 0.2883 - accuracy: 0.8820
Epoch 9/30
50/50 [==============================] - 6s 113ms/step - loss: 0.2591 - accuracy: 0.8930
Epoch 10/30
50/50 [==============================] - 6s 113ms/step - loss: 0.2977 - accuracy: 0.8790

In [46]:
import time
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1611850389.h5
